#Final Project Cleaning

Import libraries, base path is gdrive based

In [ ]:
from bs4 import BeautifulSoup
from collections import defaultdict
from os import path
import glob
import json
import pathlib as p
import pandas as pd
from google.colab import files
import pickle
import string
import statistics as stats
import numpy as np
from numpy import NaN
base_path = "/content/drive/MyDrive/DataSci112/DATTALION"

##Google Drive Scraping

In [ ]:
march_paths = []
marchstart = "/2022/MARCH 2022/MAR"
for i in range(3,32):
  if i < 10:
    i = str(i)
    march_path = marchstart + "0" + i + "/MAR0" + i + " Photo/**"
  else:
    i = str(i)
    march_path = marchstart + i + "/MAR" + i + " Photo/**"
  march_paths.append(march_path)

In [ ]:
unprocessed_path = []
exluded_months = ["FEBRUARY 2022", "MARCH 2022"]
months_2022 = ["APRIL 2022", "MAY 2022", "JUNE 2022", "JULY 2022", "AUGUST 2022", "SEPTEMBER 2022", "OCTOBER 2022", "NOVEMBER 2022", "DECEMBER 2022"]
months_2023 = ["JANUARY 2023", "FEBRUARY 2023"]
for march_path in march_paths:
  for file in glob.iglob((base_path + march_path), recursive=True):
    path = p.PurePath(file)
    #Look only at files (has a suffix)
    if path.suffix != "":
      unprocessed_path.append(path)
for month in months_2022:
  for file in glob.iglob((base_path + "/2022/" + month + "/*/**"), recursive=True):
    path = p.PurePath(file)
    #Look only at files (has a suffix)
    if path.suffix != "":
      unprocessed_path.append(path)
for month in months_2023:
  for file in glob.iglob((base_path + "/2023/" + month + "/*/**"), recursive=True):
    path = p.PurePath(file)
    #Look only at files (has a suffix)
    if path.suffix != "":
      unprocessed_path.append(path)

In [ ]:
with open("DATTALIONDATA.pkl", "wb") as outfile:
    pickle.dump(unprocessed_path, outfile)
files.download('DATTALIONDATA.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Load Files

Read in Pickle so that you don't ahve to scrape everytime

In [ ]:
with open('/content/drive/MyDrive/DataSci112/Final Project Data/DATTALIONDATA.pkl', 'rb') as f:
  pickled_paths = pickle.load(f)

In [ ]:
#Read in from GD
df_raw_cities = pd.read_csv("/content/drive/MyDrive/DataSci112/Final Project Data/ua.csv")

In [ ]:
df_cities = df_raw_cities[["city", "lat", "lng"]]
#Remove duplicate coords of big cities
df_cities = df_cities.drop([477, 1381, 1051, 21])

In [ ]:
df_cities[df_cities["city"] == "Cherkasy"]

,city,lat,lng
18,Cherkasy,49.4444,32.0597


In [ ]:
df_appended_cities = pd.DataFrame({"city": ["Chernigiv", "Zaporizhia", "Severodonetsk", "Mariupil"], "lat": [51.4982, 47.8388, 48.9482, 47.1306], "lng": [31.2893, 35.1396, 38.4879, 37.5639]})

In [ ]:
df_cities = pd.concat([df_cities, df_appended_cities], ignore_index=True)
df_cities["city"] = df_cities["city"].str.translate(str.maketrans('', '', string.punctuation))
df_cities["city"] = df_cities["city"].str.replace("ʹ", "").str.replace("’", "")
valid_cities = list(df_cities["city"])
df_cities

,city,lat,lng
0,Kyiv,50.4500,30.5236
1,Kharkiv,50.0000,36.2292
2,Odesa,46.4775,30.7326
3,Dnipro,48.4500,34.9833
4,Donetsk,48.0089,37.8042
...,...,...,...
1464,Ivanivka,46.7100,34.5500
1465,Chernigiv,51.4982,31.2893
1466,Zaporizhia,47.8388,35.1396
1467,Severodonetsk,48.9482,38.4879


In [ ]:
df_region = pd.DataFrame (data= {"region": ["Crimea","Cherkasy","Chernihiv","Chernivtsi","Dnipropetrovsk","Donetsk","IvanoFrankivsk","Kharkiv","Kherson","Khmelnytskyi","Kirovohrad","Kyiv","Luhansk","Lviv","Sevastopol","Mykolaiv","Odesa","Poltava","Rivne","Sumy","Ternopil","Vinnytsia","Volyn","Zakarpattia","Zaporizhzhia","Zhytomyr", "Ukraine"],
                                 "city": ["Simferopol","Cherkasy","Chernihiv","Chernivtsi","Dnipro","Donetsk","IvanoFrankivsk","Kharkiv","Kherson","Khmelnytskyi","Kropyvnytskyi","Kyiv","Luhansk","Lviv","Sevastopol","Mykolaiv","Odesa","Poltava","Rivne","Sumy","Ternopil","Vinnytsia","Lutsk","Uzhhorod","Zaporizhzhia","Zhytomyr", NaN]} )

In [ ]:
df_valid_region = df_region.merge(df_cities, on="city", how="left")

In [ ]:
df_valid_region.loc[26]["lng"].dtype

dtype('float64')

##Helper Functions for DataFrame Assembly

In [ ]:
def end_cleaning(title):
  if title[-2:] == "BS":
    title = title[:-3]
  if title[-1] == ")" and title[-2].isdigit():
    title = title[:-4]
  while title[-1].isdigit() or title[-1] == " " or title[-1] == "-":
    title = title[:-1]
  return title

In [ ]:
valid_months = ['MAR', 'FEB', 'APR', 'MAY', 'JUN', 'JUL','AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'JAN']
def month_cleaning(title):
  if title[:3] in valid_months:
    return title
  else:
    title = title.strip()
    if title[:3] in valid_months:
      return title
    if title[:3] == "MAE":
      title = title.replace("MAE", "MAR", 1)
      if title[:3] in valid_months:
        return title
    if title[:3] == "Mar":
      title = title.replace("Mar", "MAR", 1)
      if title[:3] in valid_months:
        return title
    if title[:3] == "ARP":
      title = title.replace("ARP", "APR", 1)
      if title[:3] in valid_months:
        return title
    if title[:3] == "DAY":
      title = title.replace("DAY", "MAY", 1)
      if title[:3] in valid_months:
        return title
    if title[:3] == "Jul":
      title = title.replace("Jul", "JUL", 1)
      if title[:3] in valid_months:
        return title
    if title[:3] == "SE0":
      title = title.replace("SE", "SEP", 1)
      if title[:3] in valid_months:
        return title
    if title[:3] == "АPR":
      title = title.replace("АPR", "APR", 1)
      if title[:3] in valid_months:
        return title
    if title[:3] == "Kha":
      title = "MAR16 " + title
      if title[:3] in valid_months:
        return title
    if title[:3] == "Che":
      title = "APR10 " + title
      if title[:3] in valid_months:
        return title
    return title

In [ ]:
valid_days = ['02', '03', '04', '25', '28', '01', '27', '26', '05', '06',
       '07', '08', '09', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23', '24', '29',
       '30', '31']
def day_cleaning(title):
  if title[3:5] in valid_days:
    return title
  else:
    if title[3:5] == " 0" or title[3:5] == " 1" or title[3:5] == " 2":
      title = title.replace(" ", "", 1)
      if title[3:5] in valid_days:
        return title
    if title[3:5] == " A":
      title = title.replace("MAR", "MAR06", 1)
      if title[3:5] in valid_days:
        return title  
    if title[3:5] == "":
      title = title.replace("MAR", "MAR19", 1)
      if title[3:5] in valid_days:
        return title
    if title[3:5] == "E0":
      title = title.replace("JUNE", "JUN", 1)
      if title[3:5] in valid_days:
        return title    
    if title[3:5] == "Y1":
      title = title.replace("JULY", "JUL", 1)
      if title[3:5] in valid_days:
        return title  
    if title[3:5] == "8 ":
      title = title.replace("8", "08", 1)
      if title[3:5] in valid_days:
        return title  
  return title

In [ ]:
# def get_region(title):
#   title = title.translate(str.maketrans('', '', string.punctuation))
#   title = title.replace("ʹ", "")
#   words = title.split(" ")
#   for word in words:
#     if word == "region":
#       i = words.index("region")
#       return words[i - 1]
#   return NaN

In [ ]:
def get_region(title):
  c_content = title.translate(str.maketrans('', '', string.punctuation))
  c_content = c_content.replace("ʹ", "").replace("’", "")
  words = title.split(" ")
  for word in words:
    region = df_valid_region[df_valid_region["region"] == word]["region"]
    if len(region) == 1:
      return region.values[0]
  return NaN

In [ ]:
# def remove_region(title, region):
#   if region == NaN:
#     return title[6:]
#   punc = title.translate(str.maketrans('', '', string.punctuation))
#   words = title.split(" ")
#   less = punc.split(" ")
#   i = less.index("region")
#   if i > len(words)//3 and i > 4:
#     return title[6:]
#   else:
#     words.pop(i)
#     words.pop(i - 1)
#     return (" ".join(words))[6:]

In [ ]:
def get_city(content, region):
  c_content = content.translate(str.maketrans('', '', string.punctuation))
  c_content = c_content.replace("ʹ", "").replace("’", "")
  words = c_content.split(" ")
  targets = []
  for word in words:
    if word in valid_cities:
      if word not in targets:
        targets.append(word)
  if not pd.isnull(region):
    if len(targets) == 0:
      return df_valid_region[df_valid_region["region"] == region]["city"].values
    i = words.index(region)
    if (i + 1) in range(len(words)):
      if region in targets and words[i + 1] == "region":
        targets.remove(region)
  if len(targets) == 0:
    return [NaN]
  return targets

In [ ]:
df_valid_region.loc[df_valid_region["region"] == "Odesa"]["lat"].values[0]

46.4775

In [ ]:
def get_best_lat(cities, region):
  lats = []
  if pd.isnull(cities).any():
    if pd.isnull(region):
      return NaN
    return df_valid_region.loc[df_valid_region["region"] == region]["lat"].values[0]
  else:
    for city in cities:
      i = valid_cities.index(city)
      lats.append(df_cities["lat"][i])
      return stats.mean(lats)

In [ ]:
def get_best_lng(cities, region):
  lngs = []
  if pd.isnull(cities).any():
    if pd.isnull(region):
      return NaN
    return df_valid_region.loc[df_valid_region["region"] == region]["lng"].values[0]
  else:
    for city in cities:
      i = valid_cities.index(city)
      lngs.append(df_cities["lng"][i])
      return stats.mean(lngs)

##Dataframe Assembly

In [ ]:
year = []
month = []
day = []
content = []
stem = []
paths = []
region = []
city = []
lat = []
lng = []
for path in pickled_paths:
  parts = list(path.parts)[5:]
  title = path.stem
  title = end_cleaning(title)
  title = month_cleaning(title)
  title = day_cleaning(title)
  if title not in stem:
    spec_region = get_region(title)
    region.append(spec_region)
    clean_content = title[6:]
    target = get_city(clean_content, spec_region)
    year.append(parts[1])
    month.append(title[:3])
    day.append(title[3:5])
    stem.append(title)
    paths.append(str(path)[43:])
    city.append(target)
    lat.append(get_best_lat(target, spec_region))
    lng.append(get_best_lng(target, spec_region))
    content.append(clean_content)

In [ ]:
print(len(year), len(month), len(day), len(content), len(paths), len(stem), len(city))

8794 8794 8794 8794 8794 8794 8794


In [ ]:
d = {"year": year, "alpha_month": month, "day": day, "city": city, "region": region, "content": content, "lat": lat, "lng": lng, "stem": stem, "path": paths}
df_DATTALION = pd.DataFrame(data = d)

In [ ]:
df_DATTALION

,year,alpha_month,day,city,region,content,lat,lng,stem,path
0,2022,MAR,02,[Kharkiv],Kharkiv,Kharkiv Missile striked school,50.0000,36.2292,MAR02 Kharkiv Missile striked school,/2022/MARCH 2022/MAR03/MAR03 Photo/MAR02 Khark...
1,2022,MAR,03,[nan],Kyiv,Hatne Kyiv region due to a powerful airstrike ...,50.4500,30.5236,MAR03 Hatne Kyiv region due to a powerful airs...,/2022/MARCH 2022/MAR03/MAR03 Photo/MAR03 Hatne...
2,2022,MAR,02,[Severodonetsk],Donetsk,Donetsk region Severodonetsk Kindergarten was ...,48.9482,38.4879,MAR02 Donetsk region Severodonetsk Kindergarte...,/2022/MARCH 2022/MAR03/MAR03 Photo/MAR02 Donet...
3,2022,MAR,03,[Kyiv],Kyiv,Kyiv russian cruise missile TAG,50.4500,30.5236,MAR03 Kyiv russian cruise missile TAG,/2022/MARCH 2022/MAR03/MAR03 Photo/MAR03 Kyiv ...
4,2022,MAR,02,[Kharkiv],Kharkiv,Kharkiv Consequences of airstrike on city hall,50.0000,36.2292,MAR02 Kharkiv Consequences of airstrike on cit...,/2022/MARCH 2022/MAR03/MAR03 Photo/MAR02 Khark...
...,...,...,...,...,...,...,...,...,...,...
8789,2023,FEB,15,[nan],Donetsk,Donetsk region Residential high-rise buildings...,48.0089,37.8042,FEB15 Donetsk region Residential high-rise bui...,/2023/FEBRUARY 2023/FEB15/FEB15 Official/FEB15...
8790,2023,FEB,15,[nan],Kharkiv,"Kharkiv region Private houses, households dama...",50.0000,36.2292,"FEB15 Kharkiv region Private houses, household...",/2023/FEBRUARY 2023/FEB15/FEB15 Official/FEB15...
8791,2023,FEB,15,[nan],Kharkiv,"Kharkiv region Private houses, households dama...",50.0000,36.2292,"FEB15 Kharkiv region Private houses, household...",/2023/FEBRUARY 2023/FEB15/FEB15 Official/FEB15...
8792,2023,FEB,15,[Pokrovsk],Donetsk,Pokrovsk Donetsk region Residential high-rise ...,48.2833,37.1833,FEB15 Pokrovsk Donetsk region Residential high...,/2023/FEBRUARY 2023/FEB15/FEB15 Official/FEB15...


In [ ]:
df_DATTALION["month"] = df_DATTALION["alpha_month"].map({
    'MAR': 3, 'FEB': 2, 'APR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10,
       'NOV': 11, 'DEC': 12, 'JAN': 1
})
df_DATTALION["date"]=pd.to_datetime(df_DATTALION[["year","month","day"]], infer_datetime_format=True)

In [ ]:
df_data = df_DATTALION[["date", "city", "region", "lat", "lng", "content"]]
df_data

,date,city,region,lat,lng,content
0,2022-03-02,[Kharkiv],Kharkiv,50.0000,36.2292,Kharkiv Missile striked school
1,2022-03-03,[nan],Kyiv,50.4500,30.5236,Hatne Kyiv region due to a powerful airstrike ...
2,2022-03-02,[Severodonetsk],Donetsk,48.9482,38.4879,Donetsk region Severodonetsk Kindergarten was ...
3,2022-03-03,[Kyiv],Kyiv,50.4500,30.5236,Kyiv russian cruise missile TAG
4,2022-03-02,[Kharkiv],Kharkiv,50.0000,36.2292,Kharkiv Consequences of airstrike on city hall
...,...,...,...,...,...,...
8789,2023-02-15,[nan],Donetsk,48.0089,37.8042,Donetsk region Residential high-rise buildings...
8790,2023-02-15,[nan],Kharkiv,50.0000,36.2292,"Kharkiv region Private houses, households dama..."
8791,2023-02-15,[nan],Kharkiv,50.0000,36.2292,"Kharkiv region Private houses, households dama..."
8792,2023-02-15,[Pokrovsk],Donetsk,48.2833,37.1833,Pokrovsk Donetsk region Residential high-rise ...


In [ ]:
df_data.iloc[195]

date                                     2022-03-08 00:00:00
city                                                [Dnipro]
region                                        Dnipropetrovsk
lat                                                    48.45
lng                                                  34.9833
content    Dnipropetrovsk region,Grigorovka after bombing...
Name: 195, dtype: object

In [ ]:
list(df_data["city"].value_counts().index)

[[nan],
 ['Kharkiv'],
 ['Mariupol'],
 ['Bakhmut'],
 ['Mykolaiv'],
 array(['Dnipro'], dtype=object),
 ['Kyiv'],
 ['Zaporizhzhia'],
 ['Kherson'],
 ['Kramatorsk'],
 ['Nikopol'],
 ['Sloviansk'],
 ['Bucha'],
 ['Kupiansk'],
 ['Irpin'],
 ['Orikhiv'],
 ['Odesa'],
 ['Chernihiv'],
 ['Avdiivka'],
 ['Zolochiv'],
 ['Lysychansk'],
 ['Izium'],
 ['Sievierodonetsk'],
 ['Kostiantynivka'],
 ['Chuhuiv'],
 ['Toretsk'],
 ['Lyman'],
 ['Rubizhne'],
 ['Pokrovsk'],
 ['Marhanets'],
 ['Komyshuvakha'],
 ['Huliaipole'],
 ['Ochakiv'],
 ['Lviv'],
 ['Druzhkivka'],
 ['Beryslav'],
 ['Polohy'],
 ['Kremenchuk'],
 ['Severodonetsk'],
 ['Stepnohirsk'],
 ['Okhtyrka'],
 ['Hostomel'],
 ['Vovchansk'],
 ['Chervonohryhorivka'],
 ['Brovary'],
 ['Makariv'],
 ['Lozova'],
 ['Enerhodar'],
 ['Mykolaivka'],
 ['Vinnytsia'],
 ['Zhytomyr'],
 ['Pervomaiske'],
 ['Shevchenkove'],
 ['Trostianets'],
 ['Zaporizhia'],
 ['Mariupil'],
 ['Oleksandrivka'],
 ['Kyiv', 'Obolon'],
 ['Zelenodolsk'],
 ['Pechenihy'],
 ['Shevchenkove', 'Kupiansk'],
 ['Chernig

In [ ]:
df_data["date"]

0      2022-03-02
1      2022-03-03
2      2022-03-02
3      2022-03-03
4      2022-03-02
          ...    
8789   2023-02-15
8790   2023-02-15
8791   2023-02-15
8792   2023-02-15
8793   2023-02-15
Name: date, Length: 8794, dtype: datetime64[ns]

In [ ]:
df_data.to_csv("112_Project_DataV2.0.csv")
# files.download("112_Project_DataV1")
df_data.to_pickle("112_Project_DataV2.0.pkl")

##JSON

In [ ]:
data_json = {}
exluded_months = ["FEBRUARY 2022", "MARCH 2022"]
months_2022 = ["APRIL 2022", "MAY 2022", "JUNE 2022", "JULY 2022", "AUGUST 2022", "SEPTEMBER 2022", "OCTOBER 2022", "NOVEMBER 2022", "DECEMBER 2022"]
months_2023 = ["JANUARY 2023", "FEBRUARY 2023"]
for month in months_2022:
  for file in glob.iglob((base_path + "/2022/" + month + "/*/**"), recursive=True):
    path = p.PurePath(file)
    #Look only at files (has a suffix)
    if path.suffix != "":
      #Splits up path to create the json
      parts = list(path.parts)[5:]
      if parts[1] not in data_json:
        data_json[parts[1]] = {}
      if parts[2] not in data_json[parts[1]]:
        data_json[parts[1]][parts[2]] = {}
      if parts[3] not in data_json[parts[1]][parts[2]]:
        data_json[parts[1]][parts[2]][parts[3]] = []
      #Filters end cleaning by 4 digit or 3 digit file extentions
      if parts[-1][-4:] != "docx" and parts[-1][-4:] != "gdoc" and parts[-1][-4:] != "jpeg":
        title = parts[-1][:-4]
        if title[-2:] == "BS":
          title = title[:-3]
        if title[-1].isdigit():
          title = title[:-2]
        if title not in data_json[parts[1]][parts[2]][parts[3]]:
          data_json[parts[1]][parts[2]][parts[3]].append(title)
      if parts[-1][-4:] == "docx" or parts[-1][-4:] == "gdoc" or parts[-1][-4:] == "jpeg":
        title = parts[-1][:-5]
        if title[-2:] == "BS":
          title = title[:-3]
        if title[-1].isdigit():
          title = title[:-2]
        if title not in data_json[parts[1]][parts[2]][parts[3]]:
          data_json[parts[1]][parts[2]][parts[3]].append(title)
for month in months_2023:
  for file in glob.iglob((base_path + "/2023/" + month + "/*/**"), recursive=True):
    path = p.PurePath(file)
    #Look only at files (has a suffix)
    if path.suffix != "":
      #Splits up path to create the json
      parts = list(path.parts)[5:]
      if parts[1] not in data_json:
        data_json[parts[1]] = {}
      if parts[2] not in data_json[parts[1]]:
        data_json[parts[1]][parts[2]] = {}
      if parts[3] not in data_json[parts[1]][parts[2]]:
        data_json[parts[1]][parts[2]][parts[3]] = []
      #Filters end cleaning by 4 digit or 3 digit file extentions
      if parts[-1][-4:] != "docx" and parts[-1][-4:] != "gdoc" and parts[-1][-4:] != "jpeg":
        title = parts[-1][:-4]
        if title[-2:] == "BS":
          title = title[:-3]
        if title[-1].isdigit():
          title = title[:-2]
        if title not in data_json[parts[1]][parts[2]][parts[3]]:
          data_json[parts[1]][parts[2]][parts[3]].append(title)
      if parts[-1][-4:] == "docx" or parts[-1][-4:] == "gdoc" or parts[-1][-4:] == "jpeg":
        title = parts[-1][:-5]
        if title[-2:] == "BS":
          title = title[:-3]
        if title[-1].isdigit():
          title = title[:-2]
        if title not in data_json[parts[1]][parts[2]][parts[3]]:
          data_json[parts[1]][parts[2]][parts[3]].append(title)
  data_json

In [ ]:
with open("DATTALIONJSON.json", "w") as outfile:
    json.dump(data_json, outfile)
files.download('DATTALIONJSON.json')

In [ ]:
path = "APR13 Bucha Kyiv region Photos of civilian cars shelled by russian military, some of cars have the inscription Children 2 BS.jpg"
path[:-4][:-3]

'APR13 Bucha Kyiv region Photos of civilian cars shelled by russian military, some of cars have the inscription Children 2'